# find the sessions that include the region of interest
Note that the function list any session that includes **any** of the regions. The funciton to list the sessions that include **all** of the regions will be added in the future.

In [1]:
from functions import  get_epoch_StimOn, get_pid_eid_pairs
from functions import monitor_job_status
import submitit
import pandas as pd 
import numpy as np
import json 
from pathlib import Path
import os
import sys
from config import  LFP_dir


# create the pid_eid_pairs
pid_eid_pairs = get_pid_eid_pairs( only_passive= False, regions= ['VISp'])
# the path to the data (based on config file) and the number of downloaded files
already_done = [p[0] for p in pid_eid_pairs if p[0] in [f.split('_')[0] for f in os.listdir(LFP_dir)]]
print(f'already done {len(already_done)}')




Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Total number of possible probe insertions: 73
PID-EID pairs saved to data/pid_eids_V1.json
the path to the data is /workspaces/onedrive1/文档/ibl_data
already done 10


# parallel submition using submitit 

In [ ]:
import submitit
import os
import json

LFP_filter_params= { 'tmin': -1, 'tmax': 1.5, 'contrasts': 'all', 'stim_side': "both",
                           'prob_left': [0.8,0.2], 'remove_first_trials_of_block': False,
                             'overwrite': False, 'save': True}
def submit_extract(pid_eid_pair):
    pid, eid = pid_eid_pair
    get_epoch_StimOn(pid, **LFP_filter_params)


already_done_pids = [f.split('_')[0] for f in os.listdir(LFP_dir) if f.endswith('.fif') ]
pid_eid_pair = [p for p in pid_eid_pairs if p[0] not in already_done_pids]
# Create an executor
executor = submitit.AutoExecutor(folder='logs')
executor.update_parameters(slurm_array_parallelism= 50 , mem_gb=10, timeout_min= 120, slurm_partition= 'CPU',  cpus_per_task= 1)
# Submit all jobs using map_array                                                                                              
all_jobs = executor.map_array(submit_extract, pid_eid_pair)
monitor_job_status(all_jobs)